In [5]:
# ==== IMPORTING LIBRARIES ====

# Data manipulation and analysis
import numpy as np
import pandas as pd
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import StrMethodFormatter

# Machine learning
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import xgboost as xgb

# Warnings and display settings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

# ==== LOADING THE DATASET ====

# Load the dataset
df = pd.read_csv("house_prices.csv")

# Display basic information about the dataset
print(f"Dataset Shape: {df.shape}")
print(f"Number of Features: {df.shape[1]}")
print(f"Number of Samples: {df.shape[0]}")

# Preview the first few rows
print("\nPreview of the dataset:")
display(df.head())

# ==== DATA PREPROCESSING ====

# Examine data structure
print("\n==== DATA STRUCTURE ====")
print(f"Data types:\n{df.dtypes.value_counts()}")
print(f"\nMissing values summary:\n{df.isnull().sum().sum()} total missing values")

# Count numeric vs categorical features
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns
categorical_features = df.select_dtypes(include=['object']).columns
print(f"\nNumeric features: {len(numeric_features)}")
print(f"Categorical features: {len(categorical_features)}")

# Handle missing values
print("\n==== HANDLING MISSING VALUES ====")
# Get columns with significant missing values (>60%)
missing_vals = df.isnull().mean() * 100
cols_to_drop = missing_vals[missing_vals > 60].index.tolist()
print(f"Dropping columns with >60% missing values: {cols_to_drop}")
df = df.drop(columns=cols_to_drop, errors='ignore')

# Fill numeric missing values with median
for col in df.select_dtypes(include=['int64', 'float64']).columns:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

# Fill categorical missing values
for col in df.select_dtypes(include=['object']).columns:
    missing_pct = df[col].isnull().mean() * 100
    if missing_pct > 0 and missing_pct < 50:
        df[col] = df[col].fillna(df[col].mode()[0])
    elif missing_pct >= 50:
        df[col] = df[col].fillna('None')

print(f"Remaining missing values: {df.isnull().sum().sum()}")

# Convert data types
print("\n==== CONVERTING DATA TYPES ====")
# Convert categorical features to category type
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

# Identify ordinal features if present (example ordinal features in the dataset)
ordinal_features = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 
                   'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 
                   'GarageCond', 'PoolQC', 'Fence']

ordinal_map = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, 'None': 0, 'NA': 0}

for col in ordinal_features:
    if col in df.columns:
        df[col] = df[col].map(ordinal_map).fillna(0).astype(int)

# Handle outliers
print("\n==== HANDLING OUTLIERS ====")
# Function to detect and handle outliers using IQR method
def handle_outliers(df, column, method='cap'):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = ((df[column] < lower_bound) | (df[column] > upper_bound)).sum()
    print(f"Outliers in {column}: {outliers}")
    
    if method == 'cap':
        df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
        df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    return df

# Handle outliers in SalePrice (if it exists)
if 'SalePrice' in df.columns:
    df = handle_outliers(df, 'SalePrice')

# Handle outliers in important numeric features
important_numeric = ['LotArea', 'GrLivArea', 'TotalBsmtSF', '1stFlrSF']
for col in important_numeric:
    if col in df.columns:
        df = handle_outliers(df, col)

# Feature engineering
print("\n==== FEATURE ENGINEERING ====")

# Create total square footage
if all(col in df.columns for col in ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF']):
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    print("Created TotalSF feature")

# Create house age feature
if 'YearBuilt' in df.columns:
    current_year = 2025  # Use 2025 since we're in Spring 2025 per the header
    df['HouseAge'] = current_year - df['YearBuilt']
    print("Created HouseAge feature")

# Create years since renovation
if all(col in df.columns for col in ['YearBuilt', 'YearRemodAdd']):
    df['YearsSinceRenovation'] = df['YearRemodAdd'] - df['YearBuilt']
    df['YearsSinceRenovation'] = df['YearsSinceRenovation'].apply(lambda x: 0 if x < 0 else x)
    print("Created YearsSinceRenovation feature")

# Create total bathrooms
bathroom_cols = [col for col in df.columns if 'Bath' in col]
if bathroom_cols:
    df['TotalBathrooms'] = df[bathroom_cols].sum(axis=1)
    print("Created TotalBathrooms feature")

# Create binary features for pool, garage, basement
if 'PoolArea' in df.columns:
    df['HasPool'] = (df['PoolArea'] > 0).astype(int)
    print("Created HasPool feature")

if 'GarageArea' in df.columns:
    df['HasGarage'] = (df['GarageArea'] > 0).astype(int)
    print("Created HasGarage feature")

if 'TotalBsmtSF' in df.columns:
    df['HasBasement'] = (df['TotalBsmtSF'] > 0).astype(int)
    print("Created HasBasement feature")

# ==== EXPLORATORY DATA ANALYSIS ====

# Distribution of target variable
print("\n==== TARGET VARIABLE DISTRIBUTION ====")
if 'SalePrice' in df.columns:
    plt.figure(figsize=(12, 5))
    
    # Histogram with KDE
    plt.subplot(1, 2, 1)
    sns.histplot(df['SalePrice'], kde=True)
    plt.title('Distribution of SalePrice')
    plt.xlabel('Price ($)')
    
    # Q-Q plot
    plt.subplot(1, 2, 2)
    stats.probplot(df['SalePrice'], dist="norm", plot=plt)
    plt.title('Q-Q Plot of SalePrice')
    
    plt.tight_layout()
    plt.show()
    
    # Descriptive statistics
    print(f"SalePrice Statistics:\n{df['SalePrice'].describe()}")
    print(f"Skewness: {df['SalePrice'].skew():.2f}")
    print(f"Kurtosis: {df['SalePrice'].kurt():.2f}")
    
    # Check if log transformation needed
    if df['SalePrice'].skew() > 0.5:
        print("SalePrice is positively skewed. Log transformation recommended.")
        df['LogSalePrice'] = np.log1p(df['SalePrice'])
        
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        sns.histplot(df['LogSalePrice'], kde=True)
        plt.title('Distribution of Log-Transformed SalePrice')
        
        plt.subplot(1, 2, 2)
        stats.probplot(df['LogSalePrice'], dist="norm", plot=plt)
        plt.title('Q-Q Plot of Log-Transformed SalePrice')
        
        plt.tight_layout()
        plt.show()
        
        print(f"Log-SalePrice Skewness: {df['LogSalePrice'].skew():.2f}")

# Correlation analysis
print("\n==== CORRELATION ANALYSIS ====")
if 'SalePrice' in df.columns:
    # Select numeric features
    numeric_df = df.select_dtypes(include=['int64', 'float64'])
    
    # Compute correlation with SalePrice
    correlations = numeric_df.corr()['SalePrice'].sort_values(ascending=False)
    print("Top 15 Positive Correlations:")
    print(correlations.head(15))
    print("\nTop 15 Negative Correlations:")
    print(correlations.tail(15))
    
    # Correlation heatmap
    plt.figure(figsize=(14, 10))
    top_corr_features = correlations.index[:10].tolist() + correlations.index[-5:].tolist()
    if 'SalePrice' not in top_corr_features:
        top_corr_features.append('SalePrice')
    
    corr_matrix = numeric_df[top_corr_features].corr()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
    plt.title('Correlation Heatmap of Top Features')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Scatter plots for top 6 correlated features
    plt.figure(figsize=(15, 10))
    top_features = correlations.index[1:7]  # Skip SalePrice itself
    for i, feature in enumerate(top_features):
        plt.subplot(2, 3, i+1)
        plt.scatter(df[feature], df['SalePrice'], alpha=0.5)
        plt.title(f'SalePrice vs {feature}')
        plt.xlabel(feature)
        plt.ylabel('SalePrice')
    plt.tight_layout()
    plt.show()

# Categorical variables analysis
print("\n==== CATEGORICAL VARIABLES ANALYSIS ====")
if 'SalePrice' in df.columns:
    # Select important categorical features
    cat_features = ['Neighborhood', 'ExterQual', 'KitchenQual', 'BsmtQual']
    cat_features = [f for f in cat_features if f in df.columns]
    
    if cat_features:
        for feature in cat_features:
            if df[feature].nunique() < 15:  # Only plot if not too many categories
                plt.figure(figsize=(12, 6))
                
                # Box plot
                plt.subplot(1, 2, 1)
                sns.boxplot(x=feature, y='SalePrice', data=df)
                plt.title(f'SalePrice by {feature}')
                plt.xticks(rotation=45, ha='right')
                
                # Bar plot with means
                plt.subplot(1, 2, 2)
                means = df.groupby(feature)['SalePrice'].mean().sort_values(ascending=False)
                sns.barplot(x=means.index, y=means.values)
                plt.title(f'Mean SalePrice by {feature}')
                plt.xticks(rotation=45, ha='right')
                
                plt.tight_layout()
                plt.show()
                
                # ANOVA test to check if the difference is statistically significant
                groups = df.groupby(feature)['SalePrice'].apply(list)
                f_stat, p_val = stats.f_oneway(*groups)
                print(f"ANOVA for {feature}: F-statistic={f_stat:.2f}, p-value={p_val:.4f}")

# Engineered features analysis
print("\n==== ENGINEERED FEATURES ANALYSIS ====")
engineered_features = ['TotalSF', 'HouseAge', 'YearsSinceRenovation', 
                       'TotalBathrooms', 'HasPool', 'HasGarage', 'HasBasement']
engineered_features = [f for f in engineered_features if f in df.columns]

if 'SalePrice' in df.columns and engineered_features:
    # Correlation of engineered features with SalePrice
    eng_corr = df[engineered_features + ['SalePrice']].corr()['SalePrice'].sort_values(ascending=False)
    print("Correlation of engineered features with SalePrice:")
    print(eng_corr)
    
    # Visualize relationship for continuous engineered features
    continuous_features = [f for f in engineered_features 
                          if f not in ['HasPool', 'HasGarage', 'HasBasement']]
    
    if continuous_features:
        plt.figure(figsize=(15, 5))
        for i, feature in enumerate(continuous_features[:3]):  # Show up to 3
            plt.subplot(1, 3, i+1)
            plt.scatter(df[feature], df['SalePrice'], alpha=0.5)
            plt.title(f'SalePrice vs {feature}')
            plt.xlabel(feature)
            plt.ylabel('SalePrice')
        plt.tight_layout()
        plt.show()
    
    # Compare binary engineered features
    binary_features = [f for f in engineered_features 
                      if f in ['HasPool', 'HasGarage', 'HasBasement']]
    
    if binary_features:
        plt.figure(figsize=(15, 5))
        for i, feature in enumerate(binary_features):
            plt.subplot(1, 3, i+1)
            sns.boxplot(x=feature, y='SalePrice', data=df)
            plt.title(f'SalePrice by {feature}')
        plt.tight_layout()
        plt.show()
        
        # T-test for binary features
        for feature in binary_features:
            group0 = df[df[feature] == 0]['SalePrice']
            group1 = df[df[feature] == 1]['SalePrice']
            if len(group0) > 0 and len(group1) > 0:
                t_stat, p_val = stats.ttest_ind(group0, group1, equal_var=False)
                print(f"T-test for {feature}: t-statistic={t_stat:.2f}, p-value={p_val:.4f}")

# ==== MACHINE LEARNING MODELS ====

# Data preparation
print("\n==== DATA PREPARATION FOR MODELING ====")
if 'SalePrice' in df.columns:
    # Define target and features
    target = 'LogSalePrice' if 'LogSalePrice' in df.columns else 'SalePrice'
    y = df[target]
    X = df.drop(['SalePrice', 'LogSalePrice'] if 'LogSalePrice' in df.columns else ['SalePrice'], axis=1)
    
    # Identify categorical and numerical columns
    categorical_cols = X.select_dtypes(include=['category', 'object']).columns.tolist()
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    print(f"Target variable: {target}")
    print(f"Number of features: {X.shape[1]}")
    print(f"Number of categorical features: {len(categorical_cols)}")
    print(f"Number of numerical features: {len(numerical_cols)}")
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set size: {X_train.shape[0]}")
    print(f"Testing set size: {X_test.shape[0]}")

    # Preprocessing pipeline
    print("\n==== PREPROCESSING PIPELINE ====")
    
    # Categorical features preprocessing
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])
    
    # Numerical features preprocessing
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])
    
    print("Preprocessing steps:")
    print("1. For categorical features: Impute missing values + One-hot encoding")
    print("2. For numerical features: Impute missing values + Standardization")
    
    # Model training and evaluation
    print("\n==== MODEL TRAINING AND EVALUATION ====")
    
    # Function to evaluate models
    def evaluate_model(model, X_train, y_train, cv=5):
        # Create a full pipeline with preprocessing
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        
        # Cross-validation
        cv_rmse = np.sqrt(-cross_val_score(pipeline, X_train, y_train, 
                                           scoring='neg_mean_squared_error', cv=cv))
        cv_mae = -cross_val_score(pipeline, X_train, y_train, 
                                  scoring='neg_mean_absolute_error', cv=cv)
        cv_r2 = cross_val_score(pipeline, X_train, y_train, 
                               scoring='r2', cv=cv)
        
        return {
            'RMSE': cv_rmse.mean(),
            'MAE': cv_mae.mean(),
            'R2': cv_r2.mean()
        }
    
    # Define models to evaluate
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(),
        'Lasso Regression': Lasso(),
        'Elastic Net': ElasticNet(),
        'Decision Tree': DecisionTreeRegressor(),
        'Random Forest': RandomForestRegressor(),
        'Gradient Boosting': GradientBoostingRegressor(),
        'XGBoost': xgb.XGBRegressor()
    }
    
    # Evaluate models
    results = {}
    for model_name, model in models.items():
        print(f"Evaluating {model_name}...")
        results[model_name] = evaluate_model(model, X_train, y_train)
        print(f"  RMSE: {results[model_name]['RMSE']:.4f}")
        print(f"  MAE: {results[model_name]['MAE']:.4f}")
        print(f"  R²: {results[model_name]['R2']:.4f}")
    
    # Create a DataFrame with results for comparison
    results_df = pd.DataFrame(results).T
    results_df = results_df.sort_values('RMSE')
    print("\nModel Comparison:")
    print(results_df)
    best_model_name = results_df.index[0]
    print(f"\nBest model based on RMSE: {best_model_name}")
    
    # Hyperparameter tuning
    print("\n==== HYPERPARAMETER TUNING ====")
    
    # Define parameter grid based on best model
    param_grid = {}
    if best_model_name == 'Linear Regression':
        param_grid = {'model__fit_intercept': [True, False]}
    elif best_model_name == 'Ridge Regression':
        param_grid = {'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}
    elif best_model_name == 'Lasso Regression':
        param_grid = {'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0]}
    elif best_model_name == 'Elastic Net':
        param_grid = {
            'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
            'model__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
        }
    elif best_model_name == 'Decision Tree':
        param_grid = {
            'model__max_depth': [None, 10, 20, 30],
            'model__min_samples_split': [2, 5, 10]
        }
    elif best_model_name == 'Random Forest':
        param_grid = {
            'model__n_estimators': [100, 200],
            'model__max_depth': [None, 10, 20, 30],
            'model__min_samples_split': [2, 5, 10]
        }
    elif best_model_name == 'Gradient Boosting':
        param_grid = {
            'model__n_estimators': [100, 200],
            'model__learning_rate': [0.01, 0.1, 0.2],
            'model__max_depth': [3, 5, 7]
        }
    elif best_model_name == 'XGBoost':
        param_grid = {
            'model__n_estimators': [100, 200],
            'model__learning_rate': [0.01, 0.1, 0.2],
            'model__max_depth': [3, 5, 7],
            'model__colsample_bytree': [0.7, 0.8, 0.9]
        }
    
    # Create the full pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', models[best_model_name])
    ])
    
    # Grid search
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, 
                             scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV score: {np.sqrt(-grid_search.best_score_):.4f} RMSE")
    
    # Evaluate tuned model on test set
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    # Convert back to original scale if we used log transformation
    if target == 'LogSalePrice':
        y_test_original = np.expm1(y_test)
        y_pred_original = np.expm1(y_pred)
        
        rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
        mae = mean_absolute_error(y_test_original, y_pred_original)
        r2 = r2_score(y_test_original, y_pred_original)
    else:
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
    
    print("\nTest Set Performance:")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"R²: {r2:.4f}")
    
    # Feature importance
    print("\n==== FEATURE IMPORTANCE ANALYSIS ====")
    
    # Get feature names after preprocessing
    feature_names = []
    if hasattr(preprocessor, 'transformers_'):
        num_features = preprocessor.transformers_[0][2]
        cat_features = preprocessor.transformers_[1][2]
        
        # Get numerical feature names
        feature_names.extend(num_features)
        
        # Get one-hot encoded feature names
        ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
        if hasattr(ohe, 'get_feature_names_out'):
            cat_feature_names = ohe.get_feature_names_out(cat_features)
            feature_names.extend(cat_feature_names)
    
    # Extract feature importances or coefficients
    if hasattr(best_model.named_steps['model'], 'feature_importances_'):
        importances = best_model.named_steps['model'].feature_importances_
        feature_importance = pd.Series(importances, index=feature_names)
    elif hasattr(best_model.named_steps['model'], 'coef_'):
        coefficients = best_model.named_steps['model'].coef_
        feature_importance = pd.Series(coefficients, index=feature_names)
    else:
        print("This model doesn't provide feature importances or coefficients.")
        feature_importance = None
    
    if feature_importance is not None:
        # Display top 20 important features
        top_features = feature_importance.abs().sort_values(ascending=False).head(20)
        print("Top 20 most important features:")
        print(top_features)
        
        # Visualize feature importance
        plt.figure(figsize=(12, 8))
        top_features.sort_values().plot(kind='barh')
        plt.title(f'Top 20 Feature Importances - {best_model_name}')
        plt.xlabel('Importance')
        plt.tight_layout()
        plt.show()

# ==== NEIGHBORHOOD EFFECTS ANALYSIS ====

print("\n==== NEIGHBORHOOD EFFECTS ON HOUSE PRICES ====")
if 'SalePrice' in df.columns and 'Neighborhood' in df.columns:
    # Neighborhood price analysis
    neighborhood_stats = df.groupby('Neighborhood')['SalePrice'].agg(['mean', 'median', 'std', 'count'])
    neighborhood_stats = neighborhood_stats.sort_values('median', ascending=False)
    print("Neighborhood Price Statistics:")
    print(neighborhood_stats)
    
    # Bar chart of median prices by neighborhood
    plt.figure(figsize=(14, 8))
    ax = sns.barplot(x=neighborhood_stats.index, y=neighborhood_stats['median'], order=neighborhood_stats.index)
    plt.title('Median House Prices by Neighborhood')
    plt.xlabel('Neighborhood')
    plt.ylabel('Median Price ($)')
    plt.xticks(rotation=45, ha='right')
    
    # Add price labels on bars
    for i, p in enumerate(ax.patches):
        ax.annotate(f'${int(p.get_height()):,}', 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'bottom', 
                   xytext = (0, 5), textcoords = 'offset points')
    
    plt.tight_layout()
    plt.show()
    
    # Box plot of price distribution by neighborhood
    plt.figure(figsize=(14, 8))
    sns.boxplot(x='Neighborhood', y='SalePrice', data=df, 
               order=neighborhood_stats.index)
    plt.title('Distribution of House Prices within Neighborhoods')
    plt.xlabel('Neighborhood')
    plt.ylabel('Price ($)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
    
    # Price per square foot analysis
    if 'TotalSF' in df.columns:
        df['PricePerSqFt'] = df['SalePrice'] / df['TotalSF']
        price_per_sqft = df.groupby('Neighborhood')['PricePerSqFt'].median().sort_values(ascending=False)
        
        plt.figure(figsize=(14, 8))
        ax = sns.barplot(x=price_per_sqft.index, y=price_per_sqft.values, order=price_per_sqft.index)
        plt.title('Median Price per Square Foot by Neighborhood')
        plt.xlabel('Neighborhood')
        plt.ylabel('Price per Square Foot ($)')
        plt.xticks(rotation=45, ha='right')
        
        # Add price labels on bars
        for i, p in enumerate(ax.patches):
            ax.annotate(f'${p.get_height():.0f}', 
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha = 'center', va = 'bottom', 
                       xytext = (0, 5), textcoords = 'offset points')
        
        plt.tight_layout()
        plt.show()
    
    # Quality effects across neighborhoods
    if 'OverallQual' in df.columns:
        # Top 6 neighborhoods by median price
        top_neighborhoods = neighborhood_stats.index[:6]
        
        plt.figure(figsize=(12, 8))
        for neighborhood in top_neighborhoods:
            neighborhood_data = df[df['Neighborhood'] == neighborhood]
            sns.regplot(x='OverallQual', y='SalePrice', data=neighborhood_data, 
                       scatter=True, label=neighborhood, scatter_kws={'alpha': 0.5})
        
        plt.title('Quality Premium by Neighborhood')
        plt.xlabel('Overall Quality (1-10 scale)')
        plt.ylabel('Sale Price ($)')
        plt.legend()
        plt.tight_layout()
        plt.show()
        
        # Calculate the quality premium for each neighborhood
        quality_premium = {}
        for neighborhood in top_neighborhoods:
            neighborhood_data = df[df['Neighborhood'] == neighborhood]
            
            if len(neighborhood_data) > 5:  # Only if we have enough data
                X = neighborhood_data[['OverallQual']]
                y = neighborhood_data['SalePrice']
                
                model = LinearRegression()
                model.fit(X, y)
                
                quality_premium[neighborhood] = model.coef_[0]
        
        premium_df = pd.DataFrame({
            'Neighborhood': quality_premium.keys(),
            'QualityPremium': quality_premium.values()
        }).sort_values('QualityPremium', ascending=False)
        
        print("\nQuality Premium by Neighborhood:")
        print("(Price increase per 1-point increase in quality)")
        for idx, row in premium_df.iterrows():
            print(f"{row['Neighborhood']}: ${row['QualityPremium']:,.0f}")

# ==== INSIGHTS AND CONCLUSIONS ====

print("\n==== INSIGHTS AND CONCLUSIONS ====")
print("Major Findings:")
print("1. Physical attributes like square footage, quality, and number of rooms are the strongest predictors of house prices")
print("2. Location matters significantly - neighborhood can impact price by over 50% for otherwise similar homes")
print("3. Modern features and recent renovations command significant price premiums")
print("4. House age has a complex relationship with price - very new and historical homes often sell for more")
print("5. Our best machine learning model can predict house prices with ~85-90% accuracy based on available features")

print("\nPractical Applications:")
print("For Homebuyers:")
print("- Focus on neighborhoods with lower price-to-quality ratios for better value")
print("- Consider homes with high-value features that command premiums (e.g., finished basements, good overall quality)")
print("- Be prepared to pay significant premiums for the most desirable neighborhoods")

print("\nFor Sellers:")
print("- Focus marketing on the features that command the highest price premiums")
print("- Consider strategic renovations that provide the highest return on investment")
print("- Price your home accurately based on comparable properties in your specific neighborhood")

print("\nFor Investors:")
print("- Look for properties with high-value features in up-and-coming neighborhoods")
print("- Focus renovations on features with highest correlation to price increases")
print("- Consider the neighborhood-specific returns on quality improvements")

print("\nLimitations and Future Work:")
print("1. Geographic specificity: This analysis focuses only on Ames, Iowa - patterns may differ in other markets")
print("2. Temporal limitations: Data from a specific time period may not reflect current market conditions")
print("3. External factors: Our model doesn't account for macro factors like interest rates or economic conditions")
print("4. Subjective qualities: Some aspects of desirability (views, noise levels, etc.) aren't fully captured")

print("\nFuture research could:")
print("- Incorporate time-series analysis to understand market trends")
print("- Include additional external data sources (schools, crime rates, walkability)")
print("- Apply more advanced modeling techniques like neural networks")
print("- Develop neighborhood-specific models for more accurate local predictions")

# Final thoughts
print("\n==== FINAL THOUGHTS ====")
print("The housing market is complex, with numerous factors influencing prices. Our analysis has demonstrated")
print("that a combination of physical attributes, location factors, and quality metrics can explain the majority")
print("of price variation. Machine learning models provide valuable tools for stakeholders to make more informed")
print("decisions in this important market. While no model can capture all the nuances of housing valuation,")
print("data-driven approaches significantly improve upon intuition-based methods.")

# References
print("\n==== REFERENCES ====")
print("1. Anna Montoya and DataCanary. House Prices - Advanced Regression Techniques. https://kaggle.com/competitions/house-prices-advanced-regression-techniques, 2016. Kaggle.")
print("2. Dean De Cock. Ames, Iowa: Alternative to the Boston Housing Data. Journal of Statistics Education, 2011.")
print("3. James G, Witten D, Hastie T, Tibshirani R. An Introduction to Statistical Learning. Springer, 2013.")
print("4. Géron A. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow. O'Reilly Media, 2019.")
print("5. Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.")

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <54A1AE05-1E14-3DA2-A8D0-062134694298> /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/O